In [1]:
import pandas as pd
import ast

In [2]:
fixtures_df = pd.read_csv('youth_league_fixtures_2020_2021.csv', encoding='utf-8')

In [3]:
fixtures_df.head(3)

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,672792,NaN,UTC,2021-03-03T13:00:00+00:00,1614776400,NaN,NaN,NaN,Stadio Georgios Karaiskáki,Piraeus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,672793,NaN,UTC,2021-03-03T15:00:00+00:00,1614783600,NaN,NaN,6550.0,Stadio Ernesto Breda,Sesto San Giovanni,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,672794,NaN,UTC,2021-03-03T15:00:00+00:00,1614783600,NaN,NaN,2919.0,Estadio Jesús Navas,Sevilla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
top_scorers_df = pd.read_csv('youth_league_top_scorers_2020_2021.csv', encoding='utf-8')
top_scorers_df.head(3)

,statistics,player.id,player.name,player.firstname,player.lastname,player.age,player.birth.date,player.birth.place,player.birth.country,player.nationality,player.height,player.weight,player.injured,player.photo
0,"[{'team': {'id': 7915, 'name': 'Midtjylland U1...",161790,A. Şimşir,Doğuhan Aral,Şimşir,22,2002-06-19,NaN,Denmark,Türkiye,NaN,NaN,False,https://media.api-sports.io/football/players/1...
1,"[{'team': {'id': 7915, 'name': 'Midtjylland U1...",156536,M. Hansen,Mads Kristian,Hansen,22,2002-07-28,Ikast,Denmark,Denmark,188 cm,70 kg,False,https://media.api-sports.io/football/players/1...
2,"[{'team': {'id': 7928, 'name': 'Salzburg U19',...",271066,R. Šimić,Roko,Šimić,21,2003-09-10,Milano,Italy,Croatia,190 cm,80 kg,False,https://media.api-sports.io/football/players/2...


In [30]:
# We will need to find a way to open this column
top_scorers_df['statistics'].head(1)

0    [{'team': {'id': 7915, 'name': 'Midtjylland U1...
Name: statistics, dtype: object

In [31]:
top_scorers_df.dtypes

statistics              object
player.id                int64
player.name             object
player.firstname        object
player.lastname         object
player.age               int64
player.birth.date       object
player.birth.place      object
player.birth.country    object
player.nationality      object
player.height           object
player.weight           object
player.injured            bool
player.photo            object
dtype: object

In [32]:
# Check the data type of each element in the 'statistics' column
print(top_scorers_df['statistics'].apply(lambda x: type(x)).value_counts())

statistics
<class 'str'>    40
Name: count, dtype: int64


In [33]:
# Function to convert string representation to a Python object
def convert_to_list(data):
    try:
        return ast.literal_eval(data) if isinstance(data, str) else data
    except:
        return []  # Return an empty list if conversion fails

# Convert the 'statistics' column
top_scorers_df['statistics'] = top_scorers_df['statistics'].apply(convert_to_list)

# Expand the 'statistics' column into separate columns
if not top_scorers_df['statistics'].empty:
    statistics_expanded = pd.json_normalize(top_scorers_df['statistics'])
    top_scorers_df = top_scorers_df.drop(columns=['statistics']).join(statistics_expanded)

# Expand the `statistics` column
statistics_expanded = pd.json_normalize(top_scorers_df[0])

# Combine the expanded columns back to the original DataFrame
top_scorers_df = top_scorers_df.drop(columns=[0]).join(statistics_expanded)

# Display the updated DataFrame
top_scorers_df.head()


,player.id,player.name,player.firstname,player.lastname,player.age,player.birth.date,player.birth.place,player.birth.country,player.nationality,player.height,...,fouls.drawn,fouls.committed,cards.yellow,cards.yellowred,cards.red,penalty.won,penalty.commited,penalty.scored,penalty.missed,penalty.saved
0,161790,A. Şimşir,Doğuhan Aral,Şimşir,22,2002-06-19,NaN,Denmark,Türkiye,NaN,...,None,None,0,0,0,None,None,None,None,None
1,156536,M. Hansen,Mads Kristian,Hansen,22,2002-07-28,Ikast,Denmark,Denmark,188 cm,...,None,None,0,0,0,None,None,None,None,None
2,271066,R. Šimić,Roko,Šimić,21,2003-09-10,Milano,Italy,Croatia,190 cm,...,None,None,1,0,0,None,None,None,None,None
3,286894,J. Gittens,Jamie Jermaine,Bynoe-Gittens,20,2004-08-08,NaN,England,England,175 cm,...,None,None,0,0,0,None,None,None,None,None
4,340569,F. Čuić,Filip,Čuić,21,2003-02-22,Split,Croatia,Bosnia and Herzegovina,NaN,...,None,None,1,0,0,None,None,None,None,None


In [34]:
top_scorers_df.loc[:5,'player.id':'goals.total']

,player.id,player.name,player.firstname,player.lastname,player.age,player.birth.date,player.birth.place,player.birth.country,player.nationality,player.height,...,games.number,games.position,games.rating,games.captain,substitutes.in,substitutes.out,substitutes.bench,shots.total,shots.on,goals.total
0,161790,A. Şimşir,Doğuhan Aral,Şimşir,22,2002-06-19,NaN,Denmark,Türkiye,NaN,...,None,Midfielder,None,False,0,1,0,None,None,7
1,156536,M. Hansen,Mads Kristian,Hansen,22,2002-07-28,Ikast,Denmark,Denmark,188 cm,...,None,Attacker,None,False,0,3,0,None,None,7
2,271066,R. Šimić,Roko,Šimić,21,2003-09-10,Milano,Italy,Croatia,190 cm,...,None,Attacker,None,False,0,3,0,None,None,6
3,286894,J. Gittens,Jamie Jermaine,Bynoe-Gittens,20,2004-08-08,NaN,England,England,175 cm,...,None,Attacker,None,False,0,4,0,None,None,6
4,340569,F. Čuić,Filip,Čuić,21,2003-02-22,Split,Croatia,Bosnia and Herzegovina,NaN,...,None,Attacker,None,False,0,4,0,None,None,6
5,335054,Luís Semedo,Luís Hemir,Silva Semedo,21,2003-08-11,Lisboa,Portugal,Portugal,192 cm,...,None,Attacker,None,False,5,4,5,None,None,5


In [35]:
# Dropping columns that we won't need
top_scorers_df.drop(columns=['games.number','games.position','games.rating','games.captain','shots.total','shots.on','player.birth.place','fouls.drawn','fouls.committed',
                             'dribbles.success','dribbles.past','player.injured','penalty.won','penalty.commited','penalty.scored','penalty.missed','penalty.saved',
                             'passes.accuracy','tackles.total','tackles.blocks','tackles.interceptions','duels.total','duels.won','dribbles.attempts',
                             'goals.conceded','goals.assists','goals.saves','passes.total','passes.key','cards.yellow','cards.yellowred','cards.red',
                             'league.flag','league.logo','substitutes.in','substitutes.out','substitutes.bench','league.country','league.id','league.name'],
                     inplace=True)
top_scorers_df.head(5)

,player.id,player.name,player.firstname,player.lastname,player.age,player.birth.date,player.birth.country,player.nationality,player.height,player.weight,player.photo,team.id,team.name,team.logo,league.season,games.appearences,games.lineups,games.minutes,goals.total
0,161790,A. Şimşir,Doğuhan Aral,Şimşir,22,2002-06-19,Denmark,Türkiye,NaN,NaN,https://media.api-sports.io/football/players/1...,7915,Midtjylland U19,https://media.api-sports.io/football/teams/791...,2021,6,6,534,7
1,156536,M. Hansen,Mads Kristian,Hansen,22,2002-07-28,Denmark,Denmark,188 cm,70 kg,https://media.api-sports.io/football/players/1...,7915,Midtjylland U19,https://media.api-sports.io/football/teams/791...,2021,4,4,313,7
2,271066,R. Šimić,Roko,Šimić,21,2003-09-10,Italy,Croatia,190 cm,80 kg,https://media.api-sports.io/football/players/2...,7928,Salzburg U19,https://media.api-sports.io/football/teams/792...,2021,9,9,769,6
3,286894,J. Gittens,Jamie Jermaine,Bynoe-Gittens,20,2004-08-08,England,England,175 cm,70 kg,https://media.api-sports.io/football/players/2...,7890,Borussia Dortmund U19,https://media.api-sports.io/football/teams/789...,2021,5,5,369,6
4,340569,F. Čuić,Filip,Čuić,21,2003-02-22,Croatia,Bosnia and Herzegovina,NaN,NaN,https://media.api-sports.io/football/players/3...,17697,Hajduk Split U19,https://media.api-sports.io/football/teams/176...,2021,5,5,414,6


In [117]:
# we have 40 rows. Let's see how many players played for two seasons
top_scorers_df.shape

(40, 19)

In [113]:
top_scorers_df

,player.id,player.name,player.firstname,player.lastname,player.age,player.birth.date,player.birth.country,player.nationality,player.height,player.weight,player.photo,team.id,team.name,team.logo,league.season,games.appearences,games.lineups,games.minutes,goals.total
0,161790,A. Şimşir,Doğuhan Aral,Şimşir,22,2002-06-19,Denmark,Türkiye,NaN,NaN,https://media.api-sports.io/football/players/1...,7915,Midtjylland U19,https://media.api-sports.io/football/teams/791...,2021,6,6,534,7
1,156536,M. Hansen,Mads Kristian,Hansen,22,2002-07-28,Denmark,Denmark,188 cm,70 kg,https://media.api-sports.io/football/players/1...,7915,Midtjylland U19,https://media.api-sports.io/football/teams/791...,2021,4,4,313,7
2,271066,R. Šimić,Roko,Šimić,21,2003-09-10,Italy,Croatia,190 cm,80 kg,https://media.api-sports.io/football/players/2...,7928,Salzburg U19,https://media.api-sports.io/football/teams/792...,2021,9,9,769,6
3,286894,J. Gittens,Jamie Jermaine,Bynoe-Gittens,20,2004-08-08,England,England,175 cm,70 kg,https://media.api-sports.io/football/players/2...,7890,Borussia Dortmund U19,https://media.api-sports.io/football/teams/789...,2021,5,5,369,6
4,340569,F. Čuić,Filip,Čuić,21,2003-02-22,Croatia,Bosnia and Herzegovina,NaN,NaN,https://media.api-sports.io/football/players/3...,17697,Hajduk Split U19,https://media.api-sports.io/football/teams/176...,2021,5,5,414,6
5,335054,Luís Semedo,Luís Hemir,Silva Semedo,21,2003-08-11,Portugal,Portugal,192 cm,NaN,https://media.api-sports.io/football/players/3...,7888,Benfica U19,https://media.api-sports.io/football/teams/788...,2021,9,4,443,5
6,336563,D. Gassama,Djeidi,Gassama,21,2003-09-10,France,France,177 cm,NaN,https://media.api-sports.io/football/players/3...,7921,PSG U19,https://media.api-sports.io/football/teams/792...,2021,8,8,688,5
7,214276,S. Diallo,Samba,Diallo,21,2003-01-05,Senegal,Senegal,168 cm,64 kg,https://media.api-sports.io/football/players/2...,7898,Dynamo Kyiv U19,https://media.api-sports.io/football/teams/789...,2021,8,8,698,5
8,161920,B. Fink,Bradley Thomas,Fink,21,2003-04-17,Switzerland,Switzerland,192 cm,87 kg,https://media.api-sports.io/football/players/1...,7890,Borussia Dortmund U19,https://media.api-sports.io/football/teams/789...,2021,8,8,699,5
9,265385,T. Jambor,Timotej,Jambor,21,2003-04-04,Slovakia,Slovakia,187 cm,NaN,https://media.api-sports.io/football/players/2...,7972,Žilina U19,https://media.api-sports.io/football/teams/797...,2021,6,5,397,5


In [51]:
top_scorers_df[top_scorers_df.duplicated(subset='player.name')] # we see only one player that played for two seasons. 

,player.id,player.name,player.firstname,player.lastname,player.age,player.birth.date,player.birth.country,player.nationality,player.height,player.weight,player.photo,team.id,team.name,team.logo,league.season,games.appearences,games.lineups,games.minutes,goals.total
30,335335,J. Rijkhoff,Julian Dean,Rijkhoff,19,2005-01-25,Netherlands,Netherlands,183 cm,NaN,https://media.api-sports.io/football/players/3...,7890,Borussia Dortmund U19,https://media.api-sports.io/football/teams/789...,2022,9,9,734,5


In [97]:
# let's begin some simple analysis. top 5 players in terms of goals scored in 2 seasons. Since we alredy know that J. Rijkhoff played for two seasons I think the better stat would be goals per min.
top_scorers_df.groupby('player.name')['goals.total'].sum().reset_index(name='total_goals').sort_values(by='total_goals', ascending= False).head(5)

,player.name,total_goals
19,J. Rijkhoff,9
6,Bilal Mazhar,9
25,M. Meerdink,9
3,Adrián Niño,8
11,E. Poku,8


In [79]:
top_scorers_grouped = top_scorers_df.groupby(['player.id','player.name'])[['goals.total', 'games.minutes']].sum().reset_index().rename(columns={'goals.total': 'total_goals', 'games.minutes': 'total_min'}).sort_values(by='total_goals', ascending=False)


In [80]:
# minutes needed for goal
top_scorers_grouped['min_for_goals'] = (top_scorers_grouped['total_min']/top_scorers_grouped['total_goals']).round(1)

In [85]:
top_scorers_grouped = top_scorers_grouped.sort_values(by = 'min_for_goals')
top_scorers_grouped.head(5)

,player.id,player.name,total_goals,total_min,min_for_goals
32,351156,Bilal Mazhar,9,325,36.1
29,342954,M. Meerdink,9,344,38.2
13,288101,O. Cannonier,7,277,39.6
0,156536,M. Hansen,7,313,44.7
1,158697,J. McAtee,5,251,50.2


In [92]:
# new we will join our values to the main table
df_goals_to_visualize = top_scorers_grouped.merge(top_scorers_df, on = 'player.id').drop(columns=['player.name_y','goals.total','games.minutes','games.lineups','games.appearences'])
df_goals_to_visualize.head(4)

,player.id,player.name_x,total_goals,total_min,min_for_goals,player.firstname,player.lastname,player.age,player.birth.date,player.birth.country,player.nationality,player.height,player.weight,player.photo,team.id,team.name,team.logo,league.season
0,351156,Bilal Mazhar,9,325,36.1,Bilal Mazhar Mohamed Abdelrahm,Abdelgalil Zeeni,21,2003-11-21,France,Egypt,183 cm,75 kg,https://media.api-sports.io/football/players/3...,20236,Panathinaikos U19,https://media.api-sports.io/football/teams/202...,2022
1,342954,M. Meerdink,9,344,38.2,Mexx,Meerdink,21,2003-07-24,Netherlands,Netherlands,182 cm,69 kg,https://media.api-sports.io/football/players/3...,16384,AZ U19,https://media.api-sports.io/football/teams/163...,2022
2,288101,O. Cannonier,7,277,39.6,Oakley William,Cannonier,20,2004-05-06,England,England,NaN,NaN,https://media.api-sports.io/football/players/2...,7909,Liverpool U19,https://media.api-sports.io/football/teams/790...,2022
3,156536,M. Hansen,7,313,44.7,Mads Kristian,Hansen,22,2002-07-28,Denmark,Denmark,188 cm,70 kg,https://media.api-sports.io/football/players/1...,7915,Midtjylland U19,https://media.api-sports.io/football/teams/791...,2021


In [121]:
df_goals_to_visualize.drop_duplicates(subset = 'player.id', inplace = True)

In [122]:
df_goals_to_visualize.to_excel('df_goals_players.xlsx', index= False)

In [84]:
# average player min_for_goals
average_player = (top_scorers_grouped['total_min'].sum()  / top_scorers_grouped['total_goals'].sum()).round(1)
average_player

99.6